### 导入的库包括用于化学信息处理的rdkit库，还有sklearn的一些库

In [12]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
from gensim.models import word2vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import precision_recall_curve,recall_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from rdkit.ML.Descriptors import MoleculeDescriptors

from rdkit.Chem import Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit import Chem 
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
import os
import math
#os.chdir('/admin/Downloads/mol2vec-master')
%cd .


C:\Users\admin\Downloads\mol2vec-master


### 第一个实验，对分子描述符提取的特征使用分类器分类，这里选用随机森林和mlp

In [3]:
test_roc1=[]
test_roc12=[]
test_prc1=[]
test_prc12=[]
def get_fps(mol):
    calc=MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    ds = np.asarray(calc.CalcDescriptors(mol))
    arr=Fingerprinter.FingerprintMol(mol)[0]
    
    return np.append(arr,ds)
#全部的特征随即森林
for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    xtrain = np.array(list(data['Descriptors']))
    xtrain=np.nan_to_num(xtrain)
    xtrain[xtrain >= np.finfo(np.float32).max]=np.finfo(np.float32).max
    ytrain= np.array(data['activity'])
   
    test =pd.read_csv('fold_'+str(i)+'/test.csv')
    test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    test['Descriptors']=test['mol'].apply(get_fps)
    xtrain=np.nan_to_num(xtrain)
    
    xtest = np.array(list(test['Descriptors']))
    xtest=np.nan_to_num(xtest)
    xtest[xtest >= np.finfo(np.float32).max] = np.finfo(np.float32).max
    ytest=test['activity']
    xtrain = StandardScaler().fit_transform(xtrain)
    xtest = StandardScaler().fit_transform(xtest)
    rf = RandomForestClassifier(max_features='auto')
    rf2=MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=10)
    rf.fit(xtrain, ytrain)
    rf2.fit(xtrain, ytrain)
    predict_prob_y = rf.predict_proba(xtest)
    predict_prob_y2 = rf2.predict_proba(xtest)
    test_roc1.append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
    test_roc12.append(metrics.roc_auc_score(ytest,predict_prob_y2[:,1]))
    fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
    fpr2, tpr2, thresholds=(precision_recall_curve(ytest,predict_prob_y2[:,1]))
    test_prc1.append(metrics.auc( tpr,fpr))
    test_prc12.append(metrics.auc( tpr2,fpr2))
    del rf
    del rf2
    

Iteration 1, loss = 0.22632262
Iteration 2, loss = 0.09703120
Iteration 3, loss = 0.06666071
Iteration 4, loss = 0.05318610
Iteration 5, loss = 0.04551671
Iteration 6, loss = 0.03998555
Iteration 7, loss = 0.03484627
Iteration 8, loss = 0.03138832
Iteration 9, loss = 0.02876949
Iteration 10, loss = 0.02590400
Iteration 11, loss = 0.02396547
Iteration 12, loss = 0.02204197
Iteration 13, loss = 0.02038440
Iteration 14, loss = 0.01884410
Iteration 15, loss = 0.01747396
Iteration 16, loss = 0.01635332
Iteration 17, loss = 0.01515711
Iteration 18, loss = 0.01422090
Iteration 19, loss = 0.01326010
Iteration 20, loss = 0.01252532
Iteration 21, loss = 0.01182362
Iteration 22, loss = 0.01107983
Iteration 23, loss = 0.01044050
Iteration 24, loss = 0.00989231
Iteration 25, loss = 0.00932359
Iteration 26, loss = 0.00887018
Iteration 27, loss = 0.00837451
Iteration 28, loss = 0.00798907
Iteration 29, loss = 0.00760758
Iteration 30, loss = 0.00725308
Iteration 31, loss = 0.00693033
Iteration 32, los

### 从打印结果中可以看到，分子描述符特征的随机森林roc，prc结果为0.81和0.45，mlp方法结果为0.79和0.44

In [4]:
print(np.mean(test_roc1))
print(np.mean(test_prc1))
print(np.mean(test_roc12))
print(np.mean(test_prc12))

0.811045867532132
0.4549249079343823
0.7930097884300603
0.4413498076293891


### 打印具体每一折的roc和prc,可以看到即使是在同一特征上两种方法在不同折的表现也不尽相同，可以参照第一折即可看到两种方法的不同

In [6]:
print(test_roc1)
print(test_prc1)
print(test_roc12)
print(test_prc12)

[0.7204081632653061, 0.9295212765957447, 0.8946700507614214, 0.682449494949495, 0.9166666666666666, 0.9048821548821548, 0.7974452554744526, 0.45169082125603865, 0.9911167512690356, 0.821608040201005]
[0.42172121040550326, 0.6071080976250148, 0.7607418971156705, 0.24633341285661292, 0.8423054699537751, 0.5420005341880342, 0.15884387351778653, 0.0037593984962406013, 0.8333333333333334, 0.13310185185185183]
[0.95, 0.9293313069908815, 0.8375634517766497, 0.6590909090909091, 0.9188034188034189, 0.6616161616161615, 0.738138686131387, 0.5458937198067633, 0.9961928934010151, 0.6934673366834171]
[0.6021910929033963, 0.904919169540045, 0.5301480156148342, 0.2736304147778606, 0.7549646686159845, 0.40302682371210286, 0.025902701140041076, 0.005263157894736842, 0.8839285714285714, 0.029523460666317808]


### 第二个实验 mol2vec与所有描述符的混合经过随机森林和mlp

In [15]:
from gensim.models import word2vec
test_roc2=[]
test_prc2=[]
test_roc22=[]
test_prc22=[]
#mol2vec和全部descriptor的混合
model = word2vec.Word2Vec.load('./mol2vec-master/examples/models/model_300dim.pkl')
for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
    
    X_mol = np.array([x.vec for x in data['mol2vec']])
    X_mol = pd.DataFrame(X_mol)

    ytrain=data['activity']
    xtrain=data.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_de = np.array([x for x in data['Descriptors']])
    X_de = pd.DataFrame(X_de)
    xtrain = pd.concat((X_de, X_mol), axis=1)
    xtrain=np.nan_to_num(xtrain)
    test =pd.read_csv('fold_'+str(i)+'/test.csv')
    test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    test['Descriptors']=test['mol'].apply(get_fps)
    test['Descriptors'].fillna(test.mean())
    test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
    ytest=test['activity']
    xtest=test.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_mol2 = np.array([x.vec for x in test['mol2vec']])
    X_mol2 = pd.DataFrame(X_mol2)
    X_de2 = np.array([x for x in test['Descriptors']])
    X_de2 = pd.DataFrame(X_de2)
    xtest = pd.concat((X_de2, X_mol2), axis=1)
    xtest=np.nan_to_num(xtest)
    X_train = StandardScaler().fit_transform(xtrain)
    X_test = StandardScaler().fit_transform(xtest)
    #lr = BlendEnsemble(verbose=2)
    lr2=MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1)
    lr = RandomForestClassifier(max_features='auto')
    lr.fit(X_train, ytrain)
    lr2.fit(X_train, ytrain)
    predict_prob_y = lr.predict_proba(X_test)
    predict_prob_y2 = lr2.predict_proba(X_test)
    test_roc2.append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
    test_roc22.append(metrics.roc_auc_score(ytest,predict_prob_y2[:,1]))
    fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
    fpr2, tpr2, thresholds=(precision_recall_curve(ytest,predict_prob_y2[:,1]))
    test_prc2.append(metrics.auc( tpr,fpr))
    test_prc22.append(metrics.auc(tpr2,fpr2))
    
    del lr

Iteration 1, loss = 0.16041792
Iteration 2, loss = 0.06183708
Iteration 3, loss = 0.04667468
Iteration 4, loss = 0.03592574
Iteration 5, loss = 0.02878874
Iteration 6, loss = 0.02385688
Iteration 7, loss = 0.01938355
Iteration 8, loss = 0.01706065
Iteration 9, loss = 0.01479958
Iteration 10, loss = 0.01265891
Iteration 11, loss = 0.01142854
Iteration 12, loss = 0.01001006
Iteration 13, loss = 0.00906774
Iteration 14, loss = 0.00826396
Iteration 15, loss = 0.00710570
Iteration 16, loss = 0.00698685
Iteration 17, loss = 0.00619541
Iteration 18, loss = 0.00590374
Iteration 19, loss = 0.00550175
Iteration 20, loss = 0.00482865
Iteration 21, loss = 0.00476402
Iteration 22, loss = 0.00447284
Iteration 23, loss = 0.00432566
Iteration 24, loss = 0.00395242
Iteration 25, loss = 0.00367340
Iteration 26, loss = 0.00399336
Iteration 27, loss = 0.00388948
Iteration 28, loss = 0.00354819
Iteration 29, loss = 0.00342317
Iteration 30, loss = 0.00315880
Iteration 31, loss = 0.00270436
Iteration 32, los

In [16]:
print(np.mean(test_roc2))
print(np.mean(test_prc2))
print(np.mean(test_roc22))
print(np.mean(test_prc22))


0.8020580292519908
0.3954048849638902
0.7638728763498511
0.4542323642203514


### 第三个实验，不同分子指纹的分类效果，使用随机森林和mlp作为验证

In [ ]:
#各类指纹功效
import multiprocessing
from joblib import Parallel, delayed

roc = {}
prc={}     
   
for f in fps:

    roc[f] = {}
    prc[f]={}
    for m in models:
        roc[f][m]=[]
        prc[f][m]=[]
        for i in range(10):
            data =pd.read_csv('fold_'+str(i)+'/train.csv')
            data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
            test =pd.read_csv('fold_'+str(i)+'/test.csv')
            test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
            fps = {"ECFP4": data['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048)),
           "ECFP6": data['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=3, nBits=2048)),
           "RDKFP":data['mol'].apply(lambda m: AllChem.RDKFingerprint(m, fpSize=2048))}


            ytrain=np.array(data['activity'])
            fpstest = {"ECFP4": test['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048)),
           "ECFP6": test['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=3, nBits=2048)),
           "RDKFP":test['mol'].apply(lambda m: AllChem.RDKFingerprint(m, fpSize=2048))}
            xtrain = np.array(fps[f].tolist())
            xtest = np.array(fpstest[f].tolist())
            # Default models
            models = {"rf": RandomForestClassifier(max_features='auto'),
              "nnet":MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1) }
            ytest=np.array(test['activity'])

            models[m].fit(xtrain, ytrain)
            #scores[f][m + "_r2_train"] = models[m].score(X_train, y_train)
            predict_prob_y = models[m].predict_proba(xtest)
            roc[f][m].append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
            fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
            prc[f][m].append(metrics.auc( tpr,fpr))

for i in models:
    for j in fps:
        roc[i][j]=np.mean(roc[i][j])
        prc[j][i]=np.mean(prc[j][i])

### 以下打印的是三种指纹对应的随机森林和多层感知机的分类结果

In [647]:
print(roc)
print(prc)

             rf      nnet
ECFP4   0.77559  0.694632
ECFP6  0.768347  0.711039
RDKFP  0.774321  0.721036
             rf      nnet
ECFP4  0.415972  0.330636
ECFP6  0.395199  0.315473
RDKFP  0.397939  0.398258


### 第四个实验混合mol2vec和所有描述符和指纹进行分类

In [620]:
from mlens.ensemble import SuperLearner,BlendEnsemble
from sklearn.metrics import roc_auc_score,accuracy_score
test_auc4=[]
test_prc4=[]
#mol2vec和全部descriptor和指纹的混合

for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
    data['fingerprint']=data['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048))
    xf=np.array(data['fingerprint'].tolist())
    xf=pd.DataFrame(xf)
    X_mol = np.array([x.vec for x in data['mol2vec']])
    X_mol = pd.DataFrame(X_mol)
    
    ytrain=data['activity']
    xtrain=data.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_de = np.array([x for x in data['Descriptors']])
    X_de = pd.DataFrame(X_de)
    xtrain = pd.concat((X_de, X_mol,xf), axis=1)
    xtrain=np.nan_to_num(xtrain)
    test =pd.read_csv('fold_'+str(i)+'/test.csv')
    test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    test['Descriptors']=test['mol'].apply(get_fps)
    test['Descriptors'].fillna(test.mean())
    test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
    test['fingerprint']=test['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048))
    xf2=np.array(test['fingerprint'].tolist())
    xf2=pd.DataFrame(xf2)
    ytest=test['activity']
    xtest=test.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_mol2 = np.array([x.vec for x in test['mol2vec']])
    X_mol2 = pd.DataFrame(X_mol2)
    X_de2 = np.array([x for x in test['Descriptors']])
    X_de2 = pd.DataFrame(X_de2)
    xtest = pd.concat((X_de2, X_mol2,xf2), axis=1)
    xtest=np.nan_to_num(xtest)
    X_train = StandardScaler().fit_transform(xtrain)
    X_test = StandardScaler().fit_transform(xtest)
    #lr = BlendEnsemble(verbose=2)
    #lr.add([RandomForestClassifier(max_features='auto'),MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1)],)
    #lr.add_meta(LogisticRegression(solver='lbfgs', multi_class='auto'))
    #lr=MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1)
    lr = RandomForestClassifier(max_features='auto')
    lr.fit(X_train, ytrain)
    predict_prob_y = lr.predict_proba(X_test)
    test_auc4.append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
    fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
    test_prc4.append(metrics.auc( tpr,fpr))
    
    del lr

### 可以看到混合了所有特征后的模型准确度最高的，达到85%的roc值

In [636]:
print(np.mean(test_auc4))
print(np.mean(test_prc4))

0.8502830356668817
0.43819971569910193


### 第四个实验，不同采样结果加随机森林分类器的对比


In [ ]:
#欠采样对比
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import (ClusterCentroids, RandomUnderSampler,
                                     NearMiss,
                                     InstanceHardnessThreshold,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     RepeatedEditedNearestNeighbours,
                                     AllKNN,
                                     NeighbourhoodCleaningRule,
                                     OneSidedSelection)
test_auc5={}
test_prc5={}
#mol2vec和全部descriptor的混合的平衡采样
sampling={}
for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
    
    X_mol = np.array([x.vec for x in data['mol2vec']])
    X_mol = pd.DataFrame(X_mol)

    ytrain=np.array(data['activity'])
    xtrain=np.zeros([X_mol.shape[0],279])
    for p in range(X_mol.shape[0]):
        xtrain[p,:]=data['Descriptors'][p]
    
    xtrain = np.concatenate((xtrain, X_mol), axis=1)
    xtrain=np.nan_to_num(xtrain)
    xtrain[xtrain >= np.finfo(np.float32).max]=np.finfo(np.float32).max
    xtrain=np.array(xtrain)
    xtrain.shape
    sampling={'ClusterCentroids':ClusterCentroids().fit_resample(xtrain, ytrain),'RandomUnderSampler':RandomUnderSampler().fit_resample(xtrain, ytrain),
             'NearMiss':NearMiss().fit_resample(xtrain, ytrain),'InstanceHardnessThreshold':InstanceHardnessThreshold().fit_resample(xtrain, ytrain),
              'CondensedNearestNeighbour':CondensedNearestNeighbour().fit_resample(xtrain, ytrain),'SMOTE':SMOTE().fit_resample(xtrain, ytrain)
             }
    test_auc5[str(i)]=[]
    test_prc5[str(i)]=[]
    for s in sampling:
        
        X_resampled, Y_resampled = sampling[s]
        test =pd.read_csv('fold_'+str(i)+'/test.csv')
        test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
        test['Descriptors']=test['mol'].apply(get_fps)
        test['Descriptors'].fillna(test.mean())
        test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
        test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
        ytest=test['activity']
        xtest=test.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
        X_mol2 = np.array([x.vec for x in test['mol2vec']])
        X_mol2 = pd.DataFrame(X_mol2)
        X_de2 = np.array([x for x in test['Descriptors']])
        X_de2 = pd.DataFrame(X_de2)
        xtest = pd.concat((X_de2, X_mol2), axis=1)
        xtest=np.nan_to_num(xtest)

        X_train = StandardScaler().fit_transform(X_resampled)
        X_test = StandardScaler().fit_transform(xtest)

        lr = RandomForestClassifier(max_features='auto')
        lr.fit(X_train,Y_resampled)
        predict_prob_y = lr.predict_proba(X_test)
        test_auc5[str(i)].append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
        fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
        test_prc5[str(i)].append(metrics.auc( tpr,fpr))
        
test_auc5

avroc=np.zeros(6)
pvroc=np.zeros(6)
for j in range(6):
    for i in range(10):
        avroc[j]+=test_auc5[str(i)][j]
        pvroc[j]+=test_prc5[str(i)][j]


In [651]:
print(avroc*0.1)
print(pvroc*0.1)

[0.52863248 0.7603706  0.61471868 0.76443984 0.7965451  0.75578389]
[0.04164929 0.32034436 0.09140746 0.09645921 0.40275834 0.35208798]


### 第五个实验不同采样方法加xgboost分类器
由于在上面的采样实验中，所有采样方法配合随机森林使用，都没有超过未采样结果0.80，所以在这里我换用xgboost分类器再次尝试

In [35]:
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek

from imblearn.under_sampling import (NearMiss)
test_auc5={}
test_prc5={}
#mol2vec和全部descriptor的混合的平衡采样
sampling={}
for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
    
    X_mol = np.array([x.vec for x in data['mol2vec']])
    X_mol = pd.DataFrame(X_mol)

    ytrain=np.array(data['activity'])
    xtrain=np.zeros([X_mol.shape[0],279])
    for p in range(X_mol.shape[0]):
        xtrain[p,:]=data['Descriptors'][p]
    
    xtrain = np.concatenate((xtrain, X_mol), axis=1)
    xtrain=np.nan_to_num(xtrain)
    xtrain[xtrain >= np.finfo(np.float32).max]=np.finfo(np.float32).max
    xtrain=np.array(xtrain)
    xtrain.shape
    sampling={'SMOTE': SMOTE(), 
              'ADASYN':  ADASYN(random_state=42), 
              'NearMiss':NearMiss(version=3), 'SMOTE+ENN':  SMOTEENN(), 'SMOTE+Tomek':  SMOTETomek()
             }
    test_auc5[str(i)]=[]
    test_prc5[str(i)]=[]
    for s in sampling:
        
        X_resampled, Y_resampled = sampling[s].fit_resample(xtrain, ytrain)
        test =pd.read_csv('fold_'+str(i)+'/test.csv')
        test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
        test['Descriptors']=test['mol'].apply(get_fps)
        test['Descriptors'].fillna(test.mean())
        test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
        test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
        ytest=test['activity']
        xtest=test.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
        X_mol2 = np.array([x.vec for x in test['mol2vec']])
        X_mol2 = pd.DataFrame(X_mol2)
        X_de2 = np.array([x for x in test['Descriptors']])
        X_de2 = pd.DataFrame(X_de2)
        xtest = pd.concat((X_de2, X_mol2), axis=1)
        xtest=np.nan_to_num(xtest)

        X_train = StandardScaler().fit_transform(X_resampled)
        X_test = StandardScaler().fit_transform(xtest)

        lr = XGBClassifier(random_state = 42, n_jobs = -1)
        lr.fit(X_train,Y_resampled)
        predict_prob_y = lr.predict_proba(X_test)
        test_auc5[str(i)].append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
        fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
        test_prc5[str(i)].append(metrics.auc( tpr,fpr))
        
test_auc5

avroc=np.zeros(5)
pvroc=np.zeros(5)
for j in range(5):
    for i in range(10):
        avroc[j]+=test_auc5[str(i)][j]
        pvroc[j]+=test_prc5[str(i)][j]


### 从结果看到尽管换用xgboost分类器，但是结果还是普遍下降（xgboost不采样roc=0.81），这里让我非常疑惑，尽管在这里的两种方法都没有使得roc结果变好，但是在我们组的图网络中采样结果却让结果变好

In [36]:
print(avroc*0.1)
print(pvroc*0.1)

[0.72078436 0.69827815 0.64164957 0.698253   0.71224579]
[0.23505263 0.27909201 0.09452962 0.24142565 0.20777271]


### 第六个实验集成投票法，从第一个实验中就可以看到，即使用了同样的特征在不同的分类方法下每折的表现也十分不同，所以尝试使用投票法对结果进行估计

In [72]:
from mlens.ensemble import SuperLearner,BlendEnsemble
from sklearn.metrics import roc_auc_score,accuracy_score
test_auc6=[]
test_prc6=[]
#mol2vec和全部descriptor和指纹的混合

for i in range(10):
    data =pd.read_csv('fold_'+str(i)+'/train.csv')
    data['mol'] = data['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    data['Descriptors']=data['mol'].apply(get_fps)
    data['Descriptors'].fillna(data.mean())
    data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
    data['fingerprint']=data['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048))
    xf=np.array(data['fingerprint'].tolist())
    xf=pd.DataFrame(xf)
    X_mol = np.array([x.vec for x in data['mol2vec']])
    X_mol = pd.DataFrame(X_mol)
    
    ytrain=data['activity']
    xtrain=data.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_de = np.array([x for x in data['Descriptors']])
    X_de = pd.DataFrame(X_de)
    xtrain = pd.concat((X_de, X_mol,xf), axis=1)
    xtrain=np.nan_to_num(xtrain)
    test =pd.read_csv('fold_'+str(i)+'/test.csv')
    test['mol'] = test['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 
    test['Descriptors']=test['mol'].apply(get_fps)
    test['Descriptors'].fillna(test.mean())
    test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
    test['fingerprint']=test['mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048))
    xf2=np.array(test['fingerprint'].tolist())
    xf2=pd.DataFrame(xf2)
    ytest=test['activity']
    xtest=test.drop(columns=['smiles', 'activity','mol','sentence','mol2vec'])
    X_mol2 = np.array([x.vec for x in test['mol2vec']])
    X_mol2 = pd.DataFrame(X_mol2)
    X_de2 = np.array([x for x in test['Descriptors']])
    X_de2 = pd.DataFrame(X_de2)
    xtest = pd.concat((X_de2, X_mol2,xf2), axis=1)
    xtest=np.nan_to_num(xtest)
    X_train = StandardScaler().fit_transform(xtrain)
    X_test = StandardScaler().fit_transform(xtest)
    #lr = BlendEnsemble(verbose=2)
    #lr.add([RandomForestClassifier(max_features='auto'),MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1)],)
    #lr.add_meta(LogisticRegression(solver='lbfgs', multi_class='auto'))
    lr2=MLPClassifier(activation='relu', solver='adam', alpha=0.0001,verbose=1)
    lr = RandomForestClassifier(max_features='auto')
    lr3 = XGBClassifier(random_state = 42, n_jobs = -1)
    lr.fit(X_train, ytrain)
    lr2.fit(X_train, ytrain)
    lr3.fit(X_train, ytrain)
    predict_prob_y = (lr.predict_proba(X_test)+lr2.predict_proba(X_test)+lr3.predict_proba(X_test))/3
    test_auc6.append(metrics.roc_auc_score(ytest,predict_prob_y[:,1]))
    fpr, tpr, thresholds=(precision_recall_curve(ytest,predict_prob_y[:,1]))
    test_prc6.append(metrics.auc( tpr,fpr))
    
    del lr

Iteration 1, loss = 0.45195578
Iteration 2, loss = 0.04759425
Iteration 3, loss = 0.01449684
Iteration 4, loss = 0.00806241
Iteration 5, loss = 0.00455797
Iteration 6, loss = 0.00285997
Iteration 7, loss = 0.00243215
Iteration 8, loss = 0.00198261
Iteration 9, loss = 0.00182723
Iteration 10, loss = 0.00180385
Iteration 11, loss = 0.00169753
Iteration 12, loss = 0.00162357
Iteration 13, loss = 0.00139233
Iteration 14, loss = 0.00131881
Iteration 15, loss = 0.00129868
Iteration 16, loss = 0.00118180
Iteration 17, loss = 0.00113718
Iteration 18, loss = 0.00108286
Iteration 19, loss = 0.00105293
Iteration 20, loss = 0.00111642
Iteration 21, loss = 0.00096632
Iteration 22, loss = 0.00088769
Iteration 23, loss = 0.00108628
Iteration 24, loss = 0.00081393
Iteration 25, loss = 0.00091593
Iteration 26, loss = 0.00087110
Iteration 27, loss = 0.00077899
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.61398380
Iteration 2, loss = 0.05

In [75]:
np.mean(test_prc6)

0.4650270236320745